In [1]:
import dash
from dash import dcc, html, Input, Output, callback, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration des couleurs
COLORS = {
    'primary': '#4f75aa',
    'secondary': '#083c66', 
    'light': '#cce5ff',
    'background': '#4f75aa'
}

# Initialisation de l'app avec Bootstrap
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)


# Fonction pour charger les données
def load_data():
    """Fonction pour charger les données"""
    try:
        print("Chargement des fichiers CSV...")
        
        df_previsions = pd.read_csv('previsions_models.csv')
        df_serie_complete = pd.read_csv('serie_complete.csv')
        df_serie_train = pd.read_csv('serie_train.csv')
        df_serie_test = pd.read_csv('serie_test.csv')
        
        # Conversion des dates
        df_serie_complete['date'] = pd.to_datetime(df_serie_complete['date'])
        df_serie_train['date'] = pd.to_datetime(df_serie_train['date'])
        df_serie_test['date'] = pd.to_datetime(df_serie_test['date'])
        df_previsions['date'] = pd.to_datetime(df_previsions['date'])
        
        print("Conversion des dates réussie!")
        
        return df_previsions, df_serie_complete, df_serie_train, df_serie_test
    
    except FileNotFoundError:
        print("Fichiers CSV non trouvés, génération de données simulées...")
        

# Chargement des données
df_previsions, df_serie_complete, df_serie_train, df_serie_test = load_data()

# Fonctions utilitaires
def calculate_metrics(y_true, y_pred):
    """Calcul des métriques de performance"""
    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return mae, rmse, mape



# Layout de l'application avec fond bleu uniforme
app.layout = html.Div([
    dbc.Container([
        # En-tête
        dbc.Row([
            dbc.Col([
                html.H1("🦠 Tableau de Bord - Prévisions de la Grippe", className="text-center my-4")
            ])
        ], style={"background-color": COLORS['primary'], "color": "white", "padding": "10px", "margin-bottom": "30px", "borderRadius": "15px"}),
        
        # Navigation
        dbc.Row([
    dbc.Col([
        dbc.ButtonGroup([
            dbc.Button("📊 Vue d'ensemble", id='tab-overview', className="me-2"),
            dbc.Button("🎯 Performance des Modèles", id='tab-models', color="outline-light")
        ], className="d-flex justify-content-center w-100")
    ])
], className="mb-4"),
        
        # Contenu des onglets
        html.Div(id='tab-content')
        
    ], fluid=True, className="p-4")
], style={"backgroundColor": COLORS['background'], "minHeight": "100vh"})

# Callbacks pour la navigation
@app.callback(
    [Output('tab-overview', 'color'),
     Output('tab-models', 'color'),
     Output('tab-content', 'children')],
    [Input('tab-overview', 'n_clicks'),
     Input('tab-models', 'n_clicks')]
)
def update_tabs(overview_clicks, models_clicks):
    ctx = dash.callback_context
    if not ctx.triggered:
        return 'light', 'outline-light', create_overview_tab()
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if button_id == 'tab-overview':
        return 'light', 'outline-light', create_overview_tab()
    else:
        return 'outline-light', 'light', create_models_tab()

def create_overview_tab():
    """Création de l'onglet vue d'ensemble"""
    return [
        # Filtres
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader(html.H4("Filtres", className="text-center"),
                                   style={"backgroundColor": COLORS['light'], "color": COLORS['secondary']}),
                    dbc.CardBody([
                        dbc.Row([
                            dbc.Col([
                                dbc.Label("Années à afficher:"),
                                dcc.Dropdown(
                                    id='year-dropdown',
                                    options=
                                            [{'label': str(year), 'value': year} for year in sorted(df_serie_complete['date'].dt.year.unique())],
                            
                                    multi=True,
                                    clearable=False
                                ),
                            ], md=12),
                        ])
                    ], style={"backgroundColor": "white"})
                ], style={"backgroundColor": "white"})
            ])
        ], className="mb-4"),
        
        # Graphique principal avec hauteur augmentée
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader(html.H4("Évolution des Cas de Grippe", className="text-center"),
                                   style={"backgroundColor": COLORS['light'], "color": COLORS['secondary']}),
                    dbc.CardBody([
                        dcc.Graph(
                            id='main-timeseries',
                            style={'height': '600px'}  # Hauteur augmentée
                        )
                    ], style={"backgroundColor": "white"})
                ], style={"backgroundColor": "white"})
            ], md=12),
        ], className="mb-4"),

    ]

def create_models_tab():
    """Création de l'onglet performance des modèles"""
    model_columns = [col for col in df_previsions.columns if col != 'date' and col != 'Realisee']
    
    return [
        # Filtres
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader(html.H4("Sélection des Modèles", className="text-center"),
                                   style={"backgroundColor": COLORS['light'], "color": COLORS['secondary']}),
                    dbc.CardBody([
                        dbc.Row([
                            dbc.Col([
                                dbc.Label("Modèles à comparer:"),
                                dcc.Dropdown(
                                    id='models-dropdown',
                                    options=[{'label': model, 'value': model} for model in model_columns],
                                    value=[],  # Aucun modèle sélectionné par défaut
                                    multi=True,
                                    placeholder="Sélectionnez les modèles à afficher",
                                    clearable=True
                                ),
                            ], md=12),
                        ])
                    ], style={"backgroundColor": "white"})
                ], style={"backgroundColor": "white"})
            ])
        ], className="mb-4"),
        
        # Graphique et métriques avec hauteur augmentée
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader(html.H4("Prévisions vs Observés", className="text-center"),
                                   style={"backgroundColor": COLORS['light'], "color": COLORS['secondary']}),
                    dbc.CardBody([
                        dcc.Graph(
                            id='predictions-vs-observed',
                            style={'height': '600px'}  # Hauteur augmentée
                        )
                    ], style={"backgroundColor": "white"})
                ], style={"backgroundColor": "white"})
            ], md=8),
            
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader(html.H4("Métriques de Performance", className="text-center"),
                                   style={"backgroundColor": COLORS['light'], "color": COLORS['secondary']}),
                    dbc.CardBody([
                        html.Div(id='metrics-table')
                    ], style={"backgroundColor": "white"})
                ], className="h-100", style={"backgroundColor": "white"})
            ], md=4),
        ]),
    ]

# Callbacks pour l'onglet vue d'ensemble
@app.callback(
    Output('main-timeseries', 'figure'),
    Input('year-dropdown', 'value')
)
def update_serie_complete(annees_selectionnees):
    fig = go.Figure()

    # Si 'all' est sélectionné ou si rien n'est sélectionné (par sécurité)
    if not annees_selectionnees or 'all' in annees_selectionnees:
        total_cas = df_serie_complete.iloc[:, 1].sum()
        fig.add_trace(go.Scatter(
            x=df_serie_complete.iloc[:, 0],
            y=df_serie_complete.iloc[:, 1],
            mode='lines',
            name=f'Série complète (Total: {total_cas:.0f})',
            line=dict(color='blue')
        ))
        fig.update_layout(
            title="Évolution du Syndrome Grippal",
            xaxis_title="Date",
            yaxis_title="Nombre de cas",
            hovermode='x unified',
            height=550,  # Hauteur du graphique augmentée
            plot_bgcolor='white',
            paper_bgcolor='white',
            xaxis=dict(showgrid=True, gridcolor='#e5ecf6'),
            yaxis=dict(showgrid=True, gridcolor='#e5ecf6')
        )
        return fig

    # Sinon, on affiche par année sélectionnée
    colors = px.colors.qualitative.Set1
    for i, annee in enumerate(annees_selectionnees):
        data_annee = df_serie_complete[df_serie_complete['date'].dt.year == annee]
        total_annee = data_annee['valeur'].sum()
        fig.add_trace(go.Scatter(
            x=data_annee['date'].dt.month,
            y=data_annee['valeur'],
            mode='lines+markers',
            name=f'{annee} (Total: {total_annee:.0f})',
            line=dict(color=colors[i % len(colors)])
        ))
    fig.update_xaxes(
        showgrid=True,
        gridcolor='#e5ecf6',
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=['Jan', 'Fév', 'Mar', 'Avr', 'Mai', 'Jun',
                  'Jul', 'Aoû', 'Sep', 'Oct', 'Nov', 'Déc']
    )
    fig.update_yaxes(
        showgrid=True,
        gridcolor='#e5ecf6'
    )

    fig.update_xaxes(
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=['Jan', 'Fév', 'Mar', 'Avr', 'Mai', 'Jun',
                  'Jul', 'Aoû', 'Sep', 'Oct', 'Nov', 'Déc']
    )
    fig.update_layout(
        title="Évolution du Syndrome Grippal",
        xaxis_title="Mois",
        yaxis_title="Nombre de cas",
        hovermode='x unified',
        height=550,  # Hauteur du graphique augmentée
        plot_bgcolor='white',
        paper_bgcolor='white'
    )

    return fig

# Callbacks pour l'onglet modèles
@app.callback(
    Output('predictions-vs-observed', 'figure'),
    Input('models-dropdown', 'value')
)
def update_predictions_vs_observed(selected_models):
    try:
        fig = go.Figure()
        
        # Toujours afficher les données observées
        fig.add_trace(go.Scatter(
            x=df_serie_test['date'],
            y=df_serie_test['valeur'],
            mode='lines+markers',
            name='Observé',
            line=dict(color='black', width=3, dash = "dash"),
            marker=dict(size=8)
        ))
        
        # Ajouter les prévisions seulement si des modèles sont sélectionnés
        if selected_models:
            colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
            for i, model in enumerate(selected_models):
                if model in df_previsions.columns:
                    fig.add_trace(go.Scatter(
                        x=df_previsions['date'],
                        y=df_previsions[model],
                        mode='lines+markers',
                        name=f'Prévision {model}',
                        line=dict(color=colors[i % len(colors)], width=2),
                        marker=dict(size=6)
                    ))
        
        fig.update_layout(
            title="Comparaison Prévisions vs Observations",
            xaxis_title="Date",
            yaxis_title="Nombre de Cas",
            plot_bgcolor='white',
            paper_bgcolor='white',
            font_color='black',
            title_font_size=16,
            title_x=0.5,
            height=550,  # Hauteur du graphique augmentée
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
            hovermode='x unified',
            xaxis=dict(showgrid=True, gridcolor='#e5ecf6'),
            yaxis=dict(showgrid=True, gridcolor='#e5ecf6')
        )
        
        return fig
    except Exception as e:
        print(f"Erreur dans update_predictions_vs_observed: {e}")
        fig = go.Figure()
        fig.add_annotation(
            text=f"Erreur: {str(e)}",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False,
            font=dict(size=14, color="red")
        )
        fig.update_layout(height=550, plot_bgcolor='white', paper_bgcolor='white')
        return fig

@app.callback(
    Output('metrics-table', 'children'),
    Input('models-dropdown', 'value')
)
def update_metrics_table(selected_models):
    try:
        if not selected_models:
            return html.Div("Sélectionnez des modèles pour voir les métriques", className="text-center text-muted")
        
        # Calcul des métriques pour chaque modèle
        metrics_data = []
        for model in selected_models:
            if model in df_previsions.columns:
                merged_data = pd.merge(df_serie_test, df_previsions[['date', model]], on='date', how='inner')
                
                if not merged_data.empty:
                    mae, rmse, mape = calculate_metrics(merged_data['valeur'], merged_data[model])
                    metrics_data.append({
                        'Modèle': model,
                        'MAE': f"{mae:.0f}",
                        'RMSE': f"{rmse:.0f}",
                        'MAPE (%)': f"{mape:.1f}"
                    })
        
        if not metrics_data:
            return html.Div("Aucune donnée disponible", className="text-center text-muted")
        
        # Création du tableau
        table = dash_table.DataTable(
            data=metrics_data,
            columns=[
                {"name": "Modèle", "id": "Modèle"},
                {"name": "MAE", "id": "MAE"},
                {"name": "RMSE", "id": "RMSE"},
                {"name": "MAPE (%)", "id": "MAPE (%)"}
            ],
            style_cell={
                'textAlign': 'center',
                'padding': '10px',
                'fontFamily': 'Arial',
                'fontSize': '14px'
            },
            style_header={
                'backgroundColor': COLORS['primary'],
                'color': 'white',
                'fontWeight': 'bold'
            },
            style_data={
                'backgroundColor': '#f8f9fa',
                'color': 'black'
            }
        )
        
        return table
    except Exception as e:
        print(f"Erreur dans update_metrics_table: {e}")
        return html.Div(f"Erreur: {str(e)}", className="text-center text-danger")

if __name__ == '__main__':
    app.run_server(debug=True, port=8066, jupyter_mode="external")

Chargement des fichiers CSV...
Conversion des dates réussie!
Dash app running on http://127.0.0.1:8066/
